# Proyecto individual: Sistema de recomendación de películas
​
Este proyecto constará de dos fases: `Ingenieria de datos`, `Modelamiento y evaluación con machine learning`.

### 1. Ingeniería de datos
* Esto incluye la limpieza y transformación de los datos, abordando problemas como:
    * valores faltantes, 
    * datos duplicados y variables irrelevantes,
    * valores anidados,
    * formateo de columnas,
    * nubes de palabras para ver las más frecuentes.
    * a fin de mejorar la calidad del dataset para el modelado.
    * análisis univariado.
    * análisis bivariado y multivariado.
​
### 2. Modelamiento y evaluación con machine learning
* Implementar un modelo de clasificación con aprendizaje supervisado que permita clasificar (**con un algoritmo de coseno de similitud, por ejemplo**) las películas por ... para encontrar una lista de 5 películas similares 

### 1. Análisis exploratorio de datos

#### 1.1. Importación de librerías

In [174]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import datetime

# Mostrar figuras de matplotlib en el entorno de Jupyter Notebook
%matplotlib inline

#### 1.2. Carga y visualización los datos.

In [175]:
df_movies = pd.read_csv('datasets/movies.csv')
df_movies.head(3)

C:\Users\Josue\AppData\Local\Temp\ipykernel_3628\2792449306.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('datasets/movies.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


#### 1.2. Eliminación de filas

In [176]:
# Se eliminan las películas que su idioma original no es en inglés (habria que analizar que porcentaje representan)
df_movies_1 = df_movies[df_movies['original_language'] == 'en']

In [177]:
df_movies_1 = df_movies_1[df_movies_1['video'].notna()] 
df_movies_1 = df_movies_1[df_movies_1['status'].notna()] 


#### 1.3. Eliminación de columnas 

In [178]:
df_movies_2 = df_movies_1.drop(['belongs_to_collection', 'genres', 'poster_path', 'homepage', 'imdb_id', 'overview', 'production_companies', 'production_countries', 'original_language', 'runtime', 'spoken_languages', 'tagline', 'original_title', 'video'], axis=1)


In [179]:
df_movies_2.head()

,adult,budget,id,popularity,release_date,revenue,status,title,vote_average,vote_count
0,False,30000000,862,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0
1,False,65000000,8844,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0
2,False,0,15602,11.7129,1995-12-22,0.0,Released,Grumpier Old Men,6.5,92.0
3,False,16000000,31357,3.859495,1995-12-22,81452156.0,Released,Waiting to Exhale,6.1,34.0
4,False,0,11862,8.387519,1995-02-10,76578911.0,Released,Father of the Bride Part II,5.7,173.0


#### 1.4. Distinción entre day, month y year en date

In [180]:
# Convertir la columna 'date' a datetime
df_movies_2['release_date'] = pd.to_datetime(df_movies_2['release_date'], errors='coerce')

# Verificar el formato de la columna 'date' después de la conversión
print(df_movies_2['release_date'].head())

0   1995-10-30
1   1995-12-15
2   1995-12-22
3   1995-12-22
4   1995-02-10
Name: release_date, dtype: datetime64[ns]


In [181]:
# Crear nuevas columnas para día, mes y año
df_movies_2['day'] = df_movies_2['release_date'].dt.day
df_movies_2['month'] = df_movies_2['release_date'].dt.month
df_movies_2['year'] = df_movies_2['release_date'].dt.year

In [182]:
df_movies_2.head(2)

,adult,budget,id,popularity,release_date,revenue,status,title,vote_average,vote_count,day,month,year
0,False,30000000,862,21.946943,1995-10-30,373554033.0,Released,Toy Story,7.7,5415.0,30.0,10.0,1995.0
1,False,65000000,8844,17.015539,1995-12-15,262797249.0,Released,Jumanji,6.9,2413.0,15.0,12.0,1995.0


#### 1.5. Eliminación de la columna status

In [183]:
df_movies_2 = df_movies_2.drop(['status'], axis=1)

In [184]:
df_movies_2.head(2)

,adult,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,day,month,year
0,False,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,30.0,10.0,1995.0
1,False,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,15.0,12.0,1995.0


In [185]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32199 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   adult         32199 non-null  object        
 1   budget        32199 non-null  object        
 2   id            32199 non-null  object        
 3   popularity    32199 non-null  object        
 4   release_date  32135 non-null  datetime64[ns]
 5   revenue       32199 non-null  float64       
 6   title         32199 non-null  object        
 7   vote_average  32199 non-null  float64       
 8   vote_count    32199 non-null  float64       
 9   day           32135 non-null  float64       
 10  month         32135 non-null  float64       
 11  year          32135 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(5)
memory usage: 3.2+ MB


In [186]:
df_movies_2.describe()

,release_date,revenue,vote_average,vote_count,day,month,year
count,32135,3.219900e+04,32199.000000,32199.000000,32135.000000,32135.000000,32135.000000
mean,1991-08-10 06:16:46.229967232,1.520396e+07,5.494528,141.858039,14.120492,6.451066,1991.117037
min,1878-06-14 00:00:00,0.000000e+00,0.000000,0.000000,1.000000,1.000000,1878.000000
25%,1978-04-06 00:00:00,0.000000e+00,5.000000,3.000000,6.000000,3.000000,1978.000000
50%,2000-12-22 00:00:00,0.000000e+00,5.900000,10.000000,14.000000,7.000000,2000.000000
75%,2010-10-08 12:00:00,0.000000e+00,6.700000,43.000000,22.000000,10.000000,2010.000000
max,2020-12-16 00:00:00,2.787965e+09,10.000000,14075.000000,31.000000,12.000000,2020.000000
std,NaN,7.565460e+07,1.936492,575.156215,9.259583,3.597951,24.724798


#### 1.6. Eliminación de filas con las películas anteriores a 1980, eliminando así más del 25 % de los datos

In [187]:
df_movies_2 = df_movies_2[df_movies_2['year'] >= 1990]

In [188]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20909 entries, 0 to 45465
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   adult         20909 non-null  object        
 1   budget        20909 non-null  object        
 2   id            20909 non-null  object        
 3   popularity    20909 non-null  object        
 4   release_date  20909 non-null  datetime64[ns]
 5   revenue       20909 non-null  float64       
 6   title         20909 non-null  object        
 7   vote_average  20909 non-null  float64       
 8   vote_count    20909 non-null  float64       
 9   day           20909 non-null  float64       
 10  month         20909 non-null  float64       
 11  year          20909 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(5)
memory usage: 2.1+ MB


## 2.1. Creación de la funcion para el endpoint1

In [189]:
month_mapping = {
    "enero": 1, "febrero": 2, "marzo": 3, "abril": 4,
    "mayo": 5, "junio": 6, "julio": 7, "agosto": 8,
    "septiembre": 9, "octubre": 10, "noviembre": 11, "diciembre": 12
}

In [190]:
def cantidad_filmaciones_mes(df, month: str):
    """Obtiene la cantidad de filmaciones según mes indicado
    Args:
        df: DataFrame de películas 
        month (str): nombre del mes en castellano
    Raises:
        HTTPException: en caso de que el mes indicado no sea válido o no esté escrito en castellano o sea nulo
    Returns:
        _type_: entero o integer
    """
    # Convertir el nombre del mes a su número correspondiente
    month_number = month_mapping.get(month.lower())
    if month_number is None:
        raise ValueError(f"Mes '{month}' no es válido.")
    # Filtrar las filas que corresponden al mes especificado
    filtered_df = df[df['month'] == month_number]
    # Contar los IDs en las filas filtradas
    count_ids = filtered_df['id'].count()
    return {"endpoint1": f"Fueron estrenadas {count_ids} peliculas en el mes de {month}"}

In [191]:
cantidad_filmaciones_mes(df=df_movies_2, month='enero')

{'endpoint1': 'Fueron estrenadas 2821 peliculas en el mes de enero'}

## 2.2. Creación de la funcion para el endpoint2

In [192]:
import pandas as pd

# Función para obtener el día de la semana
def obtener_dia_semana(fecha):
    dias_semana = ['lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'domingo']
    if pd.notnull(fecha):
        return dias_semana[int(fecha.day_of_week)]
    else:
        return 'fecha inválida'

# Aplicar la función a la columna 'release_date'
df_movies_2['weekday'] = df_movies_2['release_date'].apply(obtener_dia_semana)

# Se eliminan las filas con valor de celda 'fecha inválida'
df_movies_2 = df_movies_2[df_movies_2['weekday'] != 'fecha inválida']

In [193]:
df_movies_2.head(3)

,adult,budget,id,popularity,release_date,revenue,title,vote_average,vote_count,day,month,year,weekday
0,False,30000000,862,21.946943,1995-10-30,373554033.0,Toy Story,7.7,5415.0,30.0,10.0,1995.0,lunes
1,False,65000000,8844,17.015539,1995-12-15,262797249.0,Jumanji,6.9,2413.0,15.0,12.0,1995.0,viernes
2,False,0,15602,11.7129,1995-12-22,0.0,Grumpier Old Men,6.5,92.0,22.0,12.0,1995.0,viernes


In [194]:
df_movies_2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20909 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   adult         20909 non-null  object        
 1   budget        20909 non-null  object        
 2   id            20909 non-null  object        
 3   popularity    20909 non-null  object        
 4   release_date  20909 non-null  datetime64[ns]
 5   revenue       20909 non-null  float64       
 6   title         20909 non-null  object        
 7   vote_average  20909 non-null  float64       
 8   vote_count    20909 non-null  float64       
 9   day           20909 non-null  float64       
 10  month         20909 non-null  float64       
 11  year          20909 non-null  float64       
 12  weekday       20909 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(6)
memory usage: 2.2+ MB


##### 2.2.1. Aclaración
Se agrega la columna weekday al dataset porque la memoria usada no se casi incrementada, pasando de 2.1 a 2.2 mb

In [195]:
data_movies = df_movies_2.to_csv('dataset/data_movies.csv')